In [1]:
import django
import sys
import os
sys.path.append('..')

from django.conf import settings
django.setup()

from campfin_data.models import *
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
from django.db.models import Q, Sum, Count

In [2]:
from collections import OrderedDict

committees = OrderedDict()
committees['732'] = {'yes': ['CARBW  103'], 'no': ['NO732  507', 'NORTPP 501'], 'name': '732: Carbon tax'}
committees['735'] = {'yes': ['FIX DF 103', 'WASTCA 166'], 'no': [], 'name': '735: Citizens United'}
committees['1433'] = {'yes': ['RAISUW 101'], 'no': ['NO  I1 507'], 'name': '1433: Minimum wage'}
committees['1464'] = {'yes': ['INTEW  109'], 'no': ['COALSC 107', 'NO1464 005'], 'name': '1464: Campaign finance'}
committees['1491'] = {'yes': ['ALLIGR 111'], 'no': ['KN1491 052'], 'name': '1491: Guns'}
committees['1501'] = {'yes': ['CAMPPF 111'], 'no': [], 'name': '1501: ID theft/privacy'}
committees['st3'] = {'yes': ['MASSTN 101'], 'no': ['FAMIST 009', 'NO  ST 034'], 'name': 'Sound Transit 3'}

contrib_bands = OrderedDict()
output = []

for cmte in committees:
    contrib_bands[cmte] = {'name': committees[cmte]['name'], 'sides': {'yes': {}, 'no': {}}}
    for side in ['yes', 'no']:
        contribs = ScrapedContribution.objects.filter(filing_committee__filer_id__in=committees[cmte][side])
        try:
            contrib_bands[cmte]['sides'][side]['less than 100'] = float(contribs.filter(
                amount__lt=100).aggregate(Sum('amount'))['amount__sum'])
        except:
            contrib_bands[cmte]['sides'][side]['less than 100'] = 0
        try:
            contrib_bands[cmte]['sides'][side]['less than 1000'] = float(contribs.filter(
                amount__gte=100, amount__lt=1000).aggregate(Sum('amount'))['amount__sum'])
        except:
            contrib_bands[cmte]['sides'][side]['less than 1000'] = 0
        try:
            contrib_bands[cmte]['sides'][side]['less than 10000'] = float(contribs.filter(
                amount__gte=1000, amount__lt=10000).aggregate(Sum('amount'))['amount__sum'])
        except:
            contrib_bands[cmte]['sides'][side]['less than 1000'] = 0
        try:
            contrib_bands[cmte]['sides'][side]['10000 or more'] = float(contribs.filter(
                amount__gte=10000).aggregate(Sum('amount'))['amount__sum'])
        except:
            contrib_bands[cmte]['sides'][side]['less than 1000'] = 0
        
        output.append({
            'name': '%s - %s' % (committees[cmte]['name'], side.title()),
            'bands': {key: contrib_bands[cmte]['sides'][side][key] for key in contrib_bands[cmte]['sides'][side]}
        })
print output

[{'bands': {'less than 100': 45550.87, 'less than 1000': 304193.5, 'less than 10000': 375909.62, '10000 or more': 830000.0}, 'name': '732: Carbon tax - Yes'}, {'bands': {'less than 100': 220.0, 'less than 1000': 9250.0, 'less than 10000': 102000.0, '10000 or more': 1332000.0}, 'name': '732: Carbon tax - No'}, {'bands': {'less than 100': 79041.86, 'less than 1000': 139027.56, 'less than 10000': 152689.0, '10000 or more': 126191.44}, 'name': '735: Citizens United - Yes'}, {'bands': {'less than 100': 0, 'less than 1000': 0}, 'name': '735: Citizens United - No'}, {'bands': {'less than 100': 21820.27, 'less than 1000': 12705.8, 'less than 10000': 68500.0, '10000 or more': 3353000.0}, 'name': '1433: Minimum wage - Yes'}, {'bands': {'less than 100': 0, 'less than 1000': 500.0, 'less than 10000': 9000.0, '10000 or more': 60000.0}, 'name': '1433: Minimum wage - No'}, {'bands': {'less than 100': 26608.42, 'less than 1000': 16549.0, 'less than 10000': 22300.0, '10000 or more': 3949999.99}, 'name'